## Enriching NCAATourneyDetailedResults.csv with Advanced Stats

#### If the opponents of a team score only 75.2 points on average, it could be more about the pace at which the team played instead of their skill on the defensive end. 

### The given Box score numbers are an incomplete standard of a team's performance. 

**Advanced Metrics ** in basketball provide a deeper understanding of a team's performance.

**Possession** is used to normalize basketball statistics - offensive/defensive efficiency and other metrics are all based on how the possession is calculated. Team performance should be measured on a per-possession basis.  

> Possession =0.96*[(Field Goal Attempts)+(Turnovers)+0.44*(Free Throw Attempts)-(Off.Rebounds)]


##### (Notice: Possession values are not calculated by using Play-By-Play data, as it seems like they do not necessarily add up to the final stats of the game and an estimation will do just fine)

For more information [click here](https://www.nbastuffer.com/analytics101/possession/)

### Now let's add some new features that can acutally be used for predictive modelling and ranking a team.


In [30]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from math import pi
import xgboost as xgb
from sklearn import *

df = pd.read_csv('../input/RegularSeasonDetailedResults.csv')

#Points Winning/Losing Team
df['WPts'] = df.apply(lambda row: 2*row.WFGM + row.WFGM3 + row.WFTM, axis=1)
df['LPts'] = df.apply(lambda row: 2*row.LFGM + row.LFGM3 + row.LFTM, axis=1)

#Calculate Winning/losing Team Possesion Feature
wPos = df.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
lPos = df.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)
#two teams use almost the same number of possessions in a game
#(plus/minus one or two - depending on how quarters end)
#so let's just take the average
df['Pos'] = (wPos+lPos)/2

### Advanced Metrics

In [2]:
#df = df[df['DayNum']<=132]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WPts,LPts,Pos
0,2003,134,1421,92,1411,84,N,1,32,69,...,17,28,16,15,5,0,22,92,84,75.3984
1,2003,136,1112,80,1436,51,N,0,31,66,...,8,26,12,17,10,3,15,80,51,73.5552
2,2003,136,1113,84,1272,71,N,0,31,59,...,20,22,11,12,2,5,18,84,71,66.2016
3,2003,136,1141,79,1166,73,N,0,29,53,...,14,17,20,21,6,6,21,79,73,69.8304
4,2003,136,1143,76,1301,74,N,1,27,64,...,10,26,16,14,5,8,19,76,74,66.2016


In [32]:
#df['DayNum'].describe()

count    981.000000
mean     139.060143
std        4.221598
min      134.000000
25%      136.000000
50%      137.000000
75%      139.000000
max      154.000000
Name: DayNum, dtype: float64

In [3]:
#Offensive efficiency (OffRtg) = 100 x (Points / Possessions)
df['WOffRtg'] = df.apply(lambda row: 100 * (row.WPts / row.Pos), axis=1)
df['LOffRtg'] = df.apply(lambda row: 100 * (row.LPts / row.Pos), axis=1)
#Defensive efficiency (DefRtg) = 100 x (Opponent points / Opponent possessions)
df['WDefRtg'] = df.LOffRtg
df['LDefRtg'] = df.WOffRtg
#Net Rating = Off.eff - Def.eff
df['WNetRtg'] = df.apply(lambda row:(row.WOffRtg - row.WDefRtg), axis=1)
df['LNetRtg'] = df.apply(lambda row:(row.LOffRtg - row.LDefRtg), axis=1)
                         
#Assist Ratio : Percentage of team possessions that end in assists
df['WAstR'] = df.apply(lambda row: 100 * row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
df['LAstR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
#Turnover Ratio: Number of turnovers of a team per 100 possessions used.
#(TO * 100) / (FGA + (FTA * 0.44) + AST + TO
df['WTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
df['LTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
                    
#The Shooting Percentage : Measure of Shooting Efficiency (FGA/FGA3, FTA)
df['WTSP'] = df.apply(lambda row: 100 * row.WPts / (2 * (row.WFGA + 0.44 * row.WFTA)), axis=1)
df['LTSP'] = df.apply(lambda row: 100 * row.LPts / (2 * (row.LFGA + 0.44 * row.LFTA)), axis=1)
#eFG% : Effective Field Goal Percentage adjusting for the fact that 3pt shots are more valuable 
df['WeFGP'] = df.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
df['LeFGP'] = df.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1)   
#FTA Rate : How good a team is at drawing fouls.
df['WFTAR'] = df.apply(lambda row: row.WFTA / row.WFGA, axis=1)
df['LFTAR'] = df.apply(lambda row: row.LFTA / row.LFGA, axis=1)
                         
#OREB% : Percentage of team offensive rebounds
df['WORP'] = df.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
df['LORP'] = df.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#DREB% : Percentage of team defensive rebounds
df['WDRP'] = df.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
df['LDRP'] = df.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)                                      
#REB% : Percentage of team total rebounds
df['WRP'] = df.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
df['LRP'] = df.apply(lambda row: (row.LDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1) 


#### PIE : Measure of a team's performance

> *A high PIE % is highly correlated to winning. In fact, a team’s PIE rating and a team’s winning percentage correlate at an R square of .908 which indicates a "strong" correlation*

from the official site of the [NBA](https://stats.nba.com/help/glossary/)

In [4]:
df['WPIE'] = df.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
wtmp = df.apply(lambda row: row.WPts + row.WFGM + row.WFTM - row.WFGA - row.WFTA + row.WDR + 0.5*row.WOR + row.WAst +row.WStl + 0.5*row.WBlk - row.WPF - row.WTO, axis=1)
ltmp = df.apply(lambda row: row.LPts + row.LFGM + row.LFTM - row.LFGA - row.LFTA + row.LDR + 0.5*row.LOR + row.LAst +row.LStl + 0.5*row.LBlk - row.LPF - row.LTO, axis=1) 
df['WPIE'] = wtmp/(wtmp + ltmp)
df['LPIE'] = ltmp/(wtmp + ltmp)

In [5]:

#Now let's consolidate - I've literally never read a style guide in my life
dfwins = df[['Season', 'WTeamID', 'WNetRtg', 'WPIE']]
dfwins.rename(index=str, columns={"WTeamID": "TeamID","WNetRtg": "NetRtg", "WPIE": "PIE"}, inplace = True)
dflosses = df[['Season','LTeamID', 'LNetRtg','LPIE']]
dflosses.rename(index=str, columns={"LTeamID": "TeamID","LNetRtg": "NetRtg","LPIE": "PIE"}, inplace = True)

dfadvanced = dfwins.append(dflosses)
dfadvanced = dfadvanced.groupby(['Season', 'TeamID']).agg({"NetRtg":"mean", "PIE":"mean"}).reset_index()


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [6]:
#data
games = pd.read_csv('../input/NCAATourneyCompactResults.csv')


In [7]:
dfadvanced.head()

,Season,TeamID,NetRtg,PIE
0,2003,1104,-9.179297,0.442708
1,2003,1112,14.887086,0.611089
2,2003,1113,-12.691861,0.473529
3,2003,1120,3.792182,0.536723
4,2003,1122,-31.306922,0.287611


In [8]:
#Add Ids
games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
#Add Seeds
seeds = pd.read_csv('../input/NCAATourneySeeds.csv')
seeds = {'_'.join(map(str,[int(k1)+1,k2])):int(v[1:3]) for k1, v, k2 in seeds.values}
#add 2018 Seeds
seeds = {**seeds, **{k.replace('1999_','1998_'):seeds[k] for k in seeds if '1999_' in k}}

games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)


In [9]:

#Additional Features & Clean Up
games['ScoreDiff'] = games['WScore'] - games['LScore'] 
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed'] 
games = games.fillna(-1)

sdn = games.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean()


In [10]:

#Test Set
sub = pd.read_csv('../input/SampleSubmissionStage1.csv')
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub = pd.merge(sub, sdn, how='left', on=['IDTeams'])
sub['ScoreDiffNorm'] = sub['ScoreDiffNorm'].fillna(0.)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 


In [11]:
sub.head()

,ID,Pred,Season,Team1,Team2,IDTeams,IDTeam1,IDTeam2,Team1Seed,Team2Seed,ScoreDiffNorm,SeedDiff
0,2014_1107_1110,0.5,2014,1107,1110,1107_1110,2014_1107,2014_1110,15.0,0.0,0.0,15.0
1,2014_1107_1112,0.5,2014,1107,1112,1107_1112,2014_1107,2014_1112,15.0,6.0,0.0,9.0
2,2014_1107_1113,0.5,2014,1107,1113,1107_1113,2014_1107,2014_1113,15.0,0.0,0.0,15.0
3,2014_1107_1124,0.5,2014,1107,1124,1107_1124,2014_1107,2014_1124,15.0,0.0,0.0,15.0
4,2014_1107_1140,0.5,2014,1107,1140,1107_1140,2014_1107,2014_1140,15.0,0.0,0.0,15.0


In [12]:
games2 = games[['Season', 'ID', 'Team1', 'Team2', 'Team1Seed', 'Team2Seed', 'Pred']]
games2 = games2.merge(dfadvanced, left_on =['Season', 'Team1'], right_on = ['Season', 'TeamID'], how='inner')
#col = [c for c in games.columns if c not in ['ID', 'Season', 'Team1', 'Team2','IDTeams','IDTeam1','IDTeam2','Pred','DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'ScoreDiff']]
#col = ['SeedDiff']
#sub[col].columns

In [13]:
games2.drop(['TeamID'], axis=1, inplace=True)
games2.rename(index=str, columns={"NetRtg": "Team1NetRtg", "PIE": "Team1PIE"}, inplace = True)
games2 = games2.merge(dfadvanced, left_on =['Season', 'Team2'], right_on = ['Season', 'TeamID'], how='inner')
games2.drop(['TeamID'], axis=1, inplace=True)
games2.rename(index=str, columns={"NetRtg": "Team2NetRtg", "PIE": "Team2PIE"}, inplace = True)


In [14]:
games2['Team1Seed'].replace(0, 20, inplace=True)
games2['Team2Seed'].replace(0, 20, inplace=True)
#games2['SeedDiff'] = games2['Team1Seed'] - games2['Team2Seed']

games2['SeedDiff'] = games2['Team1Seed'] - games2['Team2Seed']
games2['PIEDiff'] = games2['Team1PIE'] - games2['Team2PIE']
games2['NetRtgDiff'] = games2['Team1NetRtg'] - games2['Team2NetRtg']

In [15]:

sub2 = sub[['Season', 'ID', 'Team1', 'Team2', 'Team1Seed', 'Team2Seed', 'Pred']]
sub2['Season'] = pd.to_numeric(sub2['Season'])
sub2['Team1'] = pd.to_numeric(sub2['Team1'])
sub2['Team2'] = pd.to_numeric(sub2['Team2'])
sub2 = sub2.merge(dfadvanced, left_on =['Season', 'Team1'], right_on = ['Season', 'TeamID'], how='left')
sub2.drop(['TeamID'], axis=1, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [16]:
sub2.head()

,Season,ID,Team1,Team2,Team1Seed,Team2Seed,Pred,NetRtg,PIE
0,2014,2014_1107_1110,1107,1110,15.0,0.0,0.5,-4.398205,0.508217
1,2014,2014_1107_1112,1107,1112,15.0,6.0,0.5,-4.398205,0.508217
2,2014,2014_1107_1113,1107,1113,15.0,0.0,0.5,-4.398205,0.508217
3,2014,2014_1107_1124,1107,1124,15.0,0.0,0.5,-4.398205,0.508217
4,2014,2014_1107_1140,1107,1140,15.0,0.0,0.5,-4.398205,0.508217


In [20]:
sub2.rename(index=str, columns={"NetRtg": "Team1NetRtg", "PIE": "Team1PIE"}, inplace = True)
sub2 = sub2.merge(dfadvanced, left_on =['Season', 'Team2'], right_on = ['Season', 'TeamID'], how='left')
sub2.drop(['TeamID'], axis=1, inplace=True)
sub2.rename(index=str, columns={"NetRtg": "Team2NetRtg", "PIE": "Team2PIE"}, inplace = True)
sub2['Team1Seed'].replace(0, 20, inplace=True)
sub2['Team2Seed'].replace(0, 20, inplace=True)


In [21]:
sub2['SeedDiff'] = sub2['Team1Seed'] - sub2['Team2Seed']
sub2['PIEDiff'] = sub2['Team1PIE'] - sub2['Team2PIE']
sub2['NetRtgDiff'] = sub2['Team1NetRtg'] - sub2['Team2NetRtg']

In [27]:
sub2['Team1PIE'].describe()

count    9112.000000
mean        0.446743
std         0.115952
min         0.061350
25%         0.367413
50%         0.455974
75%         0.533188
max         0.754692
Name: Team1PIE, dtype: float64

In [23]:
col = [c for c in games2.columns if c not in ['ID', 'Season', 'Team1', 'Team2','IDTeams','IDTeam1','IDTeam2','Pred','DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'ScoreDiff', 'Team1NetRtg','Team2NetRtg','Team1Seed','Team2Seed','Team1PIE','Team2PIE']]

model = xgb.XGBRegressor()
model.fit(games2[col],games2['Pred'])
sub2['Pred'] = model.predict(sub2[col])
sub2[['ID','Pred']].to_csv('rh3p_submission.csv', index=False)

In [28]:
sub2[col].head()

,SeedDiff,PIEDiff,NetRtgDiff
0,-5.0,0.446868,67.589816
1,9.0,-0.101144,-19.148904
2,-5.0,0.032381,-1.387608
3,-5.0,-0.103134,-19.625912
4,-5.0,0.242731,23.493095


In [25]:
len(sub2)

9112

# TrueSkill is a rating system based on Bayesian inference, estimating each players skill as a gaussian like Elo rating.
# See trueskill.org for more.

import pandas as pd, numpy as np
from trueskill import TrueSkill, Rating, rate_1vs1

ts = TrueSkill(draw_probability=0.01) # 0.01 is arbitary small number
beta = 25 / 6  # default value

def win_probability(p1, p2):
    delta_mu = p1.mu - p2.mu
    sum_sigma = p1.sigma * p1.sigma + p2.sigma * p2.sigma
    denom = np.sqrt(2 * (beta * beta) + sum_sigma)
    return ts.cdf(delta_mu / denom)
    
submit = pd.read_csv('../input/SampleSubmissionStage1.csv')
submit[['Season', 'Team1', 'Team2']] = submit.apply(lambda r:pd.Series([int(t) for t in r.ID.split('_')]), axis=1)

df_tour = pd.read_csv('../input/RegularSeasonCompactResults.csv')
teamIds = np.unique(np.concatenate([df_tour.WTeamID.values, df_tour.LTeamID.values]))
ratings = { tid:ts.Rating() for tid in teamIds }

def feed_season_results(season):
    print("season = {}".format(season))
    df1 = df_tour[df_tour.Season == season]
    for r in df1.itertuples():
        ratings[r.WTeamID], ratings[r.LTeamID] = rate_1vs1(ratings[r.WTeamID], ratings[r.LTeamID])

def update_pred(season):
    beta = np.std([r.mu for r in ratings.values()]) 
    print("beta = {}".format(beta))
    submit.loc[submit.Season==season, 'Pred'] = submit[submit.Season==season].apply(lambda r:win_probability(ratings[r.Team1], ratings[r.Team2]), axis=1)

for season in sorted(df_tour.Season.unique())[:-4]: # exclude last 4 years
    feed_season_results(season)

update_pred(2014)
feed_season_results(2014)
update_pred(2015)
feed_season_results(2015)
update_pred(2016)
feed_season_results(2016)
update_pred(2017)

submit.drop(['Season', 'Team1', 'Team2'], axis=1, inplace=True)
submit.to_csv('trueskill_estimation.csv', index=None)